In [142]:
!pip install tensorflow

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [10]:
import pickle

In [11]:
import tensorflow

In [12]:
#ucitavanje dataseta
data = pd.read_csv("Churn_Modelling.csv")
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
#predprocesovanje podataka
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [14]:
data.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


## Kodiranje/Pretvaranje Kategoricke Varijable u Numericku

In [16]:
#kodiranje kategorickih varijabli
l_e_g = LabelEncoder()
data["Gender"] = l_e_g.fit_transform(data["Gender"])

In [17]:
data.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


In [18]:
#one hot encoding od "Geography"
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_koder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_koder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [19]:
geo_koder = geo_koder.toarray()

In [24]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [28]:
geo_df = pd.DataFrame(geo_koder,columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

In [30]:
geo_df.head(2)

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0


In [32]:
#sjedinjavanje kolona u jedan df ali istom linijom koda izbacivanje kolone Geography unaprijed
data = pd.concat([data.drop(["Geography"],axis=1),geo_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [34]:
#sjedinjeni/sređeni i finalni niz podataka (df)
data.shape

(10000, 13)

In [36]:
#spremanje kodera 
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(l_e_g,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file)

In [42]:
#razdvajanje niza podatak na dva dijela - ovisne i neovisne kolone/osobine
X = data.drop("Exited",axis=1)
y = data["Exited"]

In [44]:
#razdvajanje podataka u treniranju i testirajuce nizove
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [47]:
#skaliranje ovih osobina
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [49]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [51]:
with open ("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [53]:
#sa ovim radom, podaci su spremi za treniranje na osnovu umjetne mreze neurona

## Implementacija ANN - Umjetne Mreže Neurona

In [55]:
import tensorflow as tf

In [57]:
#potrebne biblioteke i alati za ovaj postupak su:
from tensorflow.keras.models import Sequential
#sledeca biblioteka se odnosi na stvaranje neurona iz kutije layers i alatom dense
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [59]:
X_train.shape[1]

12

In [61]:
#sagrađivanje ANN modela
ann_model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)), #HL1 connected to I/p - Skriveni Sloj 1 povezan sa I/p
    Dense(32,activation="relu"), #Skriveni Sloj 2
    Dense(1,activation="sigmoid")
])

C:\Users\amirb\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
ann_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000001ACEEBF2CA0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [69]:
#sačinjavanje modela/kompletiranje
#model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
ann_model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

In [71]:
#uspostavljanje Tensorboard-a, zapocinjanje treniranja
#TensorBoard sluzi samo za vizualizaciju svih logovanja koje izvrsim kod treniranja nekog modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#log_dir varijabla koja osigurava da ostane zabinjezen trag svakog treniranja modela
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [77]:
#Uspostavljanje Early Stopping - sluzi za zaustavljanje treniranja ako se vrijednost gubitaka ne smanjuje
#bitni parametri su uneseni koji sluze poboljsanju funkcije ranog zaustavljanja treniranja modela...
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)




In [79]:
#TRENIRANJE MODELA
history = ann_model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8644 - loss: 0.3328 - val_accuracy: 0.8570 - val_loss: 0.3439
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8593 - loss: 0.3478 - val_accuracy: 0.8580 - val_loss: 0.3416
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8571 - loss: 0.3357 - val_accuracy: 0.8605 - val_loss: 0.3383
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8660 - loss: 0.3279 - val_accuracy: 0.8585 - val_loss: 0.3465
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8679 - loss: 0.3238 - val_accuracy: 0.8675 - val_loss: 0.3349
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8669 - loss: 0.3281 - val_accuracy: 0.8590 - val_loss: 0.3571
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8642 - loss: 0.3330 - val_accuracy: 0.8630 - val_loss: 0.3365
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8613 - loss: 0.3287 - val_accu

In [81]:
#spremanje tog ann modela kao ".h5" jer je tako kompatibilan sa keras
ann_model.save("model.h5")

In [132]:
#!pip install -U tb-nightly


In [83]:
#Ucitavanje Tensorboard ekstenzije
%load_ext tensorboard

In [89]:
#%tensorboard --logdir logs/fit

In [ ]:
#zastao na 33:00min